# OpenAI functions
This walkthrough demonstrates how to incorporate OpenAI function-calling API's in a chain.

In [1]:
from typing import Optional

from langchain.chains.openai_functions.base import create_openai_fn_chain
from langchain.prompts import ChatPromptTemplate

## Using Python functions

In [2]:
def record_person(name: str, age: int, fav_food: Optional[str]=None) -> str:
    """Record some basic identifying information about a person.
    
    Args:
        name: The person's name.
        age: The person's age in years.
        fav_food: The name of the person's favorite food.
    """
    return f"Recording person {name} of age {age} with fav food {fav_food}!"
    
    
prompt = ChatPromptTemplate.from_template("Extract information about the relevant entity: {input}")
chain = create_openai_fn_chain([record_person], prompt=prompt, verbose=True)
chain.run("Tommy was 12, and loved apple pie")



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Extract information about the relevant entity: Tommy was 12, and loved apple pie

> Finished chain.


> Entering new  chain...
Calling function record_person with arguments:
{
  "name": "Tommy",
  "age": 12,
  "fav_food": "apple pie"
}
> Finished chain.

> Finished chain.


'Recording person Tommy of age 12 with fav food apple pie!'

## Using Pydantic objects

In [3]:
from pydantic import BaseModel, Field

class RecordPerson(BaseModel):
    """Record some basic identifying information about a person."""
    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")
        
chain = create_openai_fn_chain([RecordPerson], verbose=True)
chain.run("Sally is 13 ")



> Entering new  chain...
Prompt after formatting:
Human: Sally is 13 

> Finished chain.


RecordPerson(name='Sally', age=13, fav_food=None)

## Create a Chain that runs the chosen function


In [ ]:
from typing import Any, Callable, Dict, List, Optional

from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.base import Chain
from langchain.input import get_colored_text


class FunctionExecutorChain(Chain):
    functions: Dict[str, Callable]
    output_key: str = "output"
    input_key: str = "function"

    @property
    def input_keys(self) -> List[str]:
        return [self.input_key]

    @property
    def output_keys(self) -> List[str]:
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        """Run the logic of this chain and return the output."""
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        name = inputs["function"].pop("name")
        args = inputs["function"].pop("arguments")
        _pretty_name = get_colored_text(name, "green")
        _pretty_args = get_colored_text(json.dumps(args, indent=2), "green")
        _text = f"Calling function {_pretty_name} with arguments:\n" + _pretty_args
        _run_manager.on_text(_text)
        output = self.functions[name](**args)
        return {self.output_key: output}

In [ ]:
fn_map = {
    openai_fn["name"]: fn for openai_fn, fn in zip(openai_functions, functions)
}
exec_chain = FunctionExecutorChain(functions=fn_map, verbose=verbose)
chain = SequentialChain(
    chains=[llm_chain, exec_chain],
    input_variables=llm_chain.input_keys,
    output_variables=["output"],
    verbose=verbose,
)
chain.run("Tommy was 12, and loved apple pie")